# DuckDuckGo Web Search with LangGraph

This notebook demonstrates how to integrate DuckDuckGo search capabilities with LangGraph to create an intelligent agent that can perform live web searches.

## Overview
- Use DuckDuckGo as a search tool (no API key required!)
- Create a LangGraph agent with search capabilities
- Handle multi-step reasoning with web search

## Prerequisites
You'll need an OpenAI API key (or another LLM provider supported by LangChain)

## 1. Install Required Packages

In [ ]:
# Install required packages
!pip install -q langchain langchain-openai langgraph duckduckgo-search langchain-community

## 2. Import Dependencies

In [ ]:
import os
from typing import TypedDict, Annotated
import operator

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

## 3. Set Up API Keys

In [ ]:
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Or load from environment
# from dotenv import load_dotenv
# load_dotenv()

## 4. Initialize DuckDuckGo Search Tool

In [ ]:
# Create DuckDuckGo search tool
search = DuckDuckGoSearchRun()

# Test the search tool
print("Testing DuckDuckGo search...")
result = search.run("latest AI news 2024")
print(f"\nSearch result preview: {result[:200]}...")

## 5. Define the Agent State

In [ ]:
class AgentState(TypedDict):
    """The state of our agent."""
    messages: Annotated[list[BaseMessage], operator.add]
    # You can add more state fields as needed

## 6. Create the LangGraph Agent with Search

In [ ]:
# Initialize the LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Bind the search tool to the LLM
tools = [search]
llm_with_tools = llm.bind_tools(tools)

# Define the agent node
def call_agent(state: AgentState) -> AgentState:
    """Call the LLM with tools."""
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# Define should continue function
def should_continue(state: AgentState) -> str:
    """Determine if we should continue or end."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # If there are no tool calls, we finish
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"

# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", call_agent)
workflow.add_node("tools", ToolNode(tools))

# Set entry point
workflow.set_entry_point("agent")

# Add conditional edges
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    },
)

# Add edge from tools back to agent
workflow.add_edge("tools", "agent")

# Compile the graph
app = workflow.compile()

print("✅ LangGraph agent with DuckDuckGo search created successfully!")

## 7. Visualize the Graph (Optional)

In [ ]:
# Uncomment to visualize the graph
# try:
#     from IPython.display import Image, display
#     display(Image(app.get_graph().draw_mermaid_png()))
# except Exception as e:
#     print(f"Could not visualize graph: {e}")
#     print("Install pygraphviz for visualization: pip install pygraphviz")

## 8. Example 1: Simple Web Search Query

In [ ]:
def run_agent(query: str):
    """Helper function to run the agent with a query."""
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}\n")
    
    inputs = {"messages": [HumanMessage(content=query)]}
    
    for output in app.stream(inputs):
        for key, value in output.items():
            print(f"Node '{key}':")
            if "messages" in value:
                for msg in value["messages"]:
                    if hasattr(msg, 'content') and msg.content:
                        print(f"  {msg.content}")
                    if hasattr(msg, 'tool_calls') and msg.tool_calls:
                        print(f"  Tool calls: {msg.tool_calls}")
    
    # Get final response
    final_state = app.invoke(inputs)
    final_message = final_state["messages"][-1]
    print(f"\n{'='*60}")
    print("Final Answer:")
    print(f"{'='*60}")
    print(final_message.content)
    return final_message.content

# Example 1: Current events
run_agent("What are the latest developments in artificial intelligence this week?")

## 9. Example 2: Comparative Search

In [ ]:
# Example 2: Comparison query
run_agent("Compare the latest iPhone and Samsung Galaxy flagship phones. What are the key differences?")

## 10. Example 3: Multi-Step Reasoning

In [ ]:
# Example 3: Multi-step reasoning
run_agent("Who won the latest Nobel Prize in Physics and what was their contribution?")

## 11. Interactive Mode

In [ ]:
# Interactive mode - uncomment to use
# while True:
#     user_input = input("\nYou: ")
#     if user_input.lower() in ['quit', 'exit', 'q']:
#         print("Goodbye!")
#         break
#     
#     run_agent(user_input)

## 12. Advanced: Custom Search Parameters

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults

# Create search tool with custom parameters
custom_search = DuckDuckGoSearchResults(
    num_results=5,
    output_format="list"  # Can be 'list' or 'snippet'
)

# Test custom search
results = custom_search.run("LangChain LangGraph tutorial")
print("Custom search results:")
print(results)

## 13. Advanced: Adding Memory to the Agent

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Create agent with memory
memory = MemorySaver()
app_with_memory = workflow.compile(checkpointer=memory)

# Example with conversation history
thread_id = "conversation_1"
config = {"configurable": {"thread_id": thread_id}}

# First query
print("First query with memory:")
inputs1 = {"messages": [HumanMessage(content="Search for information about Python 3.12 new features")]}
result1 = app_with_memory.invoke(inputs1, config)
print(result1["messages"][-1].content)

# Follow-up query (agent remembers previous context)
print("\n\nFollow-up query:")
inputs2 = {"messages": [HumanMessage(content="Which of those features is most useful for async programming?")]}
result2 = app_with_memory.invoke(inputs2, config)
print(result2["messages"][-1].content)

## 14. Tips and Best Practices

### Search Query Tips:
1. **Be specific**: More specific queries yield better results
2. **Use recent dates**: Include year/month for time-sensitive info
3. **Combine keywords**: Use multiple relevant keywords

### Agent Configuration:
1. **Temperature**: Use 0 for factual queries, higher for creative tasks
2. **Model selection**: GPT-4 for complex reasoning, GPT-3.5 for speed
3. **Tool usage**: Monitor tool calls to optimize performance

### DuckDuckGo Limitations:
- No API key required (great for prototyping!)
- Rate limiting may apply with heavy usage
- Results may be less comprehensive than paid search APIs
- For production, consider alternatives like Brave Search, SerpAPI, or Tavily

## 15. Troubleshooting

### Common Issues:

1. **Import errors**: Make sure all packages are installed
```bash
pip install langchain langchain-openai langgraph duckduckgo-search langchain-community
```

2. **API key errors**: Verify your OpenAI API key is set correctly

3. **Rate limiting**: If you get rate limit errors, add delays between requests

4. **Search timeouts**: DuckDuckGo may timeout on slow connections; adjust timeout settings

## Next Steps

1. **Add more tools**: Combine with calculators, code execution, etc.
2. **Implement streaming**: Stream responses for better UX
3. **Add error handling**: Implement retry logic and fallbacks
4. **Deploy**: Package as a web app with Streamlit or FastAPI
5. **Experiment**: Try different LLMs (Anthropic Claude, local models, etc.)

## Resources

- [LangChain Documentation](https://python.langchain.com/)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [DuckDuckGo Search](https://github.com/deedy5/duckduckgo_search)